## Рефакторинг исходной таблицы \ Refactoring of initial tables

В этой тетрадке мы немножко играем с данными андийских языков.

Данные взяты [отсюда](https://docs.google.com/spreadsheets/d/1wLwgyR_YBa5yuKg8ozN4aLbi4wvRCEYkwX1Xhnryuuw/edit#gid=0). (Предоставлены проектом *Field NLP - morpology*) 

Наша задача распарсить данные и преобразовать их в онтологию. Нужно преобразовать словарные вхождения из таблицы в формат RDF и модель Ontolex lemon. 

На питон существует rdf либа:

In this notebook we will transform linguistic data into RDF. 

Data was taken from [here](https://docs.google.com/spreadsheets/d/1wLwgyR_YBa5yuKg8ozN4aLbi4wvRCEYkwX1Xhnryuuw/edit#gid=0). (data was taken from another HSE project *Field NLP - morpology*) 

python rdflib install:

In [5]:
# !pip install rdflib

Defaulting to user installation because normal site-packages is not writeable
  Using cached rdflib-5.0.0-py3-none-any.whl (231 kB)
  Using cached isodate-0.6.0-py2.py3-none-any.whl (45 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


RDF состоит из триплов, триплы это субъект-предикат-объект. Типа что - как_связано - с_чем. В либе rdflib вся эта сложная структура переведена в pythonic way -- у нас собирается что-то вроде контейнера\списка с 3-х элементными кортежами внутри. 

Вот пример из [документации](https://rdflib.readthedocs.io/en/stable/gettingstarted.html):

RDF consists of so-called triples. Triples are basically objects that consist of subject-predicate-object *= what-is connected-with what*. rdflib is a pythonic way to deal with this data type. It creates kind of list of tuples of three elements inside

Below you can find an example from docs:
 
```
[
    (subject0, predicate0, object0),
    (subject1, predicate1, object1),
    ...
    (subjectN, predicateN, objectN)
 ]
```

Чтобы сразу не бросаться в огонь, я просто переведу саму таблицу с андийскими данными в pandas. А уже потом этот dataframe можно будет аккуратно переводить в rdf.

First we transform data into pandas dataframe. Then we can transform dataframe into an RDF graph.

С гугл таблицами мне помог вот этот [туториал](https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c)

Но к сожалению, это работает везде кроме колаба. Видимо, колаб не умеет поднимать локальные сервера.

Here you can find a very helpful tutorial on how to deal with google table insife ipython notebooks. [tutorial](https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c). Unfortunately, it doesn't work well with google colab.

In [1]:
import re
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle
from tqdm import tqdm

In [3]:
andi_sheet_id = "1wLwgyR_YBa5yuKg8ozN4aLbi4wvRCEYkwX1Xhnryuuw"

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# here enter the id of your google sheet
SAMPLE_SPREADSHEET_ID_input = andi_sheet_id
SAMPLE_RANGE_NAME = 'A1:S68794'

def main():
    """
    access google table with specific id through googleapi
    """
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)  # here enter the name of your downloaded JSON file
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                                range=SAMPLE_RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input:
        print('No data found.')

main()

df = pd.DataFrame(values_input[1:], columns=values_input[0])

RefreshError: ('invalid_grant: Bad Request', '{\n  "error": "invalid_grant",\n  "error_description": "Bad Request"\n}')

In [2]:
df = pd.read_csv("andic_dicts.csv", encoding='utf-8')

d:\python371\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,10,17,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,lang,id_word,id_meaning,id,lemma,ipa,morphology,bor,annotator,pos,concepticon,meaning_ru,definition,glottocode,reference,comments,borrowing_source_word,borrowing_source_comment,"74,52%",Unnamed: 19
0,botl1242,1,1,1,а:хх,NaN,NaN,NaN,George Moroz,intj,NaN,баю-бай,баю-бай,botl1242,"Saidova, Abusov 2012",NaN,NaN,NaN,4473.0,NaN
1,NaN,2,1,2,аᴴгь/а́й,NaN,NaN,NaN,George Moroz,verb,NaN,сообщать,"понуд. от аᴴгьи; 1) сообщать, доводить до свед...",botl1242,"Saidova, Abusov 2012",NaN,NaN,NaN,9449.0,NaN
2,NaN,2,2,3,аᴴгь/а́й,NaN,NaN,NaN,George Moroz,verb,NaN,объявлять,"понуд. от аᴴгьи; 1) сообщать, доводить до свед...",botl1242,"Saidova, Abusov 2012",NaN,NaN,NaN,12706.0,NaN
3,NaN,2,3,4,аᴴгь/а́й,NaN,NaN,NaN,George Moroz,verb,NaN,нанимать,"понуд. от аᴴгьи; 1) сообщать, доводить до свед...",botl1242,"Saidova, Abusov 2012",NaN,NaN,NaN,6868.0,NaN
4,NaN,2,4,5,аᴴгь/а́й,NaN,NaN,NaN,George Moroz,verb,NaN,предупреждать,"понуд. от аᴴгьи; 1) сообщать, доводить до свед...",botl1242,"Saidova, Abusov 2012",NaN,NaN,NaN,12724.0,NaN


Делаем какой-то RDF (пока не знаем какой, просто делаем, чтобы знать, что мы такое умеем)

Let us try to compose some RDF graph:

In [4]:
import rdflib
from rdflib.namespace import RDF, RDFS, OWL, DC, DCTERMS
from rdflib.term import URIRef, Literal

In [5]:
g = rdflib.Graph(identifier='http://example.org/ontolex/andi#')  # пока рандомная ссылка

In [6]:
andi = rdflib.Namespace('http://example.org/ontolex/andi#')  # Корневой Namespace с рандомной ссылкой

ontolex = rdflib.Namespace('http://www.w3.org/ns/lemon/ontolex#')  # основная модель
lexinfo = rdflib.Namespace('http://www.lexinfo.net/ontology/2.0/lexinfo#')  # обозначение морфологических тегов
lime = rdflib.Namespace('http://www.w3.org/ns/lemon/lime#')
#synsem = rdflib.Namespace('http://www.w3.org/ns/lemon/synsem#')  # синтаксис
#decomp = rdflib.Namespace('http://www.w3.org/ns/lemon/decomp#')  # синтаксис
vartrans = rdflib.Namespace('http://www.w3.org/ns/lemon/vartrans#')  # для обозначения переводов и вариантов слов
skos = rdflib.Namespace('http://www.w3.org/2004/02/skos#')

In [7]:
g.bind('ontolex', ontolex)
g.bind('lexinfo', lexinfo)
g.bind('dct', DCTERMS)  # Это для того, чтобы названия ISO языков грузились
g.bind('rdfs', RDFS)
g.bind('lime', lime)  # ??
#g.bind('synsem', synsem)
#g.bind('decomp', decomp)
g.bind('vartrans', vartrans)
g.bind('skos', skos)

g.bind('andi', andi)

lexicon_andi = URIRef(andi)

In [8]:
# Надо вот тут добавить все типы триплов, чтобы они добавились в префиксы ниже
g.set((lexicon_andi, RDF.type, lime.Lexicon))
g.set((lexicon_andi, ontolex.language, Literal("ru, ani")))
g.set((lexicon_andi, RDFS.comment, Literal("lexicon-information_name:Andi-Russian", lang="en")))

#g.set((lexicon_andi, andi, Literal("entry")))

#lime:Lexicon_Kadag1884_bbl a ontolex:Lexicon.
#lime:Lexicon_Kadag1884_bbl ontolex:language "bbl", "ka", "ru".
#lime:Lexicon_Kadag1884_bbl rdfs:comment "lexicon-information_name:Kadagidze - Batsbi-Georgian-Russian"@en.

#g.set((apics_doc, ligt.hasUtterances, apics.examples))
#g.set((apics.examples, RDF.type, ligt.InterlinearCollection))
#Probably it should also be a `dc:bibliographicCitation`
#g.set((apics.examples, RDFS.comment, Literal(apics_dataset.properties['dc:bibliographicCitation '], lang="en")))

In [9]:
# сделала пустой turtle, прикольно
print(g.serialize(format='turtle').decode('utf-8'))

@prefix andi: <http://example.org/ontolex/andi#> .
@prefix lime: <http://www.w3.org/ns/lemon/lime#> .
@prefix ontolex: <http://www.w3.org/ns/lemon/ontolex#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

andi: a lime:Lexicon ;
    rdfs:comment "lexicon-information_name:Andi-Russian"@en ;
    ontolex:language "ru, ani" .




In [14]:
# check what POS names are used in the original table
df['pos'].value_counts().index.tolist()

['noun',
 'verb',
 'adj',
 'adv',
 'noun III',
 'noun IV',
 'masdar',
 'num',
 'noun V',
 'pron',
 'intj',
 'part',
 'x',
 'noun I',
 'post',
 'particle',
 '?',
 'noun I,II',
 'masd',
 'noun II',
 'excl',
 'expression',
 'onomatope',
 'conj',
 'participle',
 'aux',
 'adverb',
 'adv?',
 'prep',
 'morph',
 'noun?',
 'noun II,IV',
 'verb/adj',
 'adj?',
 'part?',
 'adposition',
 'noun I,II,III',
 'particle?',
 'verb?',
 'pred',
 'verb/adv',
 'ayx',
 'ordinal',
 'pronoun',
 'excl?',
 'gerund?',
 'noun VI',
 'num?',
 '???',
 'expression/noun',
 'adposition?',
 'masdar?',
 '? :)',
 'союз',
 'morpheme',
 'noun I,III',
 'part, adv',
 'noun I,I',
 'expr']

In [15]:
df['morphology'].value_counts().index.tolist()

['(-лIа)',
 'no info',
 '(-ролIи)',
 '(–лъ)',
 '4',
 '(–лъилъ)',
 '(-лIи)',
 "(-а̄ри, -ва, -айа', -айе̄гье, -а̄ре)",
 '(-лIа, -би)',
 "(-ари', -ва, -а', -е̄'гье, -е'ре)",
 '(–лъ,–би)',
 '5',
 '(-лIи, -бé)',
 '(-лIа, -аби)',
 '(-лI, -бе)',
 '(–л,–лас̄,–лá,–л̄о,–лēх)',
 '(-о, -ā, -иба, -о/-ōта, -ā)',
 '(–илъ,–аби)',
 '(-0, -с, -0, -бхва, -йдйа)',
 '(-ол|, -ди)',
 '(-л|, -ди)',
 '(–лъ,–аби)',
 '(–илъ)',
 '(-л|)',
 '(-нолIи)',
 '(-áло, -áлā, -алá, -áло/-áлōта, -áлā)',
 '(-ало, -алā, -ала, -ало/-алōта, -алā)',
 '(-áлIи)',
 '(-илI, -бе)',
 '(-лъ̄илIи)',
 '(-лъ̄илIи / -лIи, -ди)',
 '(–ø,–ас̄,–бе,–б/о,–āх)',
 '(-ол|)',
 '(-о, -ā́, -ибá, -о/-ṓта, -ā́)',
 "(-а̄'ри, -ва, -айа', -а'йе̄гье, -а̄'ре)",
 '3',
 "(-ари, -ва, -а', -е̄гье, -ере)",
 '(-лъ̄илIи / -лIи)',
 '(-о, -ā, -а, -о/-ōта, -ā)',
 '(–ø,–ас̄,–бé,–б/о,–ēх)',
 '(-е, -ас, -и, -ебхва, -идйа)',
 '(–л,–лас̄,–ла,–л̄о,–лēх)',
 '(-лI, -ди)',
 '(-ийо, -ийā, -ӣба, -ийо/-ийōта, -ийā)',
 '(-лъ̄илIи, -ди)',
 '(-илIи, -абé)',
 '(-лIи / -лъ̄илIи, -ди)'

In [57]:
for lemma in tqdm(df['lemma '][1:3800]):
    morph = df[df['lemma '] == lemma]['morphology'].values[0]
    if not pd.isna(morph) and morph.startswith("("):
        print(morph.strip("()").split(", "))

100%|█████████████████████████████████████████████████████████████████████████████| 3799/3799 [00:30<00:00, 122.88it/s]


In [56]:
for lemma in tqdm(df['lemma '][1:3800]):
    pos = df[df['lemma '] == lemma]['pos'].values[0]
    if not pd.isna(pos):
        print(pos)

  0%|                                                                                         | 0/3799 [00:00<?, ?it/s]

verb
verb
verb
verb
verb
verb
verb


  0%|▏                                                                                | 7/3799 [00:00<01:01, 62.11it/s]

verb
verb
noun
noun
noun
verb


  0%|▎                                                                               | 13/3799 [00:00<01:02, 60.66it/s]

...

 99%|█████████████████████████████████████████████████████████████████████████████▌| 3779/3799 [00:58<00:00, 82.82it/s]

verb
noun
verb


100%|█████████████████████████████████████████████████████████████████████████████▊| 3789/3799 [00:58<00:00, 86.92it/s]

verb
verb
noun
noun


100%|██████████████████████████████████████████████████████████████████████████████| 3799/3799 [00:58<00:00, 90.29it/s]


In [10]:
import urllib.parse
urllib.parse.quote_plus('см_гьйᴴдаг|аги')

'%D1%81%D0%BC_%D0%B3%D1%8C%D0%B9%E1%B4%B4%D0%B4%D0%B0%D0%B3%7C%D0%B0%D0%B3%D0%B8'

In [11]:
### m a i n ANDI ###

i = 1

for lemma in tqdm(df['lemma '][1:]):
    lemma = lemma.replace(' ', '_')
    lemma = lemma.replace('/', '_')
    lemma = lemma.replace('\\', '_')
    
    entry_num = "entry_" + str(i)
    entry_form_num = "entry_" + str(i) + "_form"
    
    # LexicalEntry
    g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), RDF.type, ontolex.LexicalEntry))
    g.add((ontolex.LexicalEntry, DCTERMS.language, URIRef('http://id.loc.gov/vocabulary/iso639-3/ani')))
    g.add((ontolex.LexicalEntry, ontolex.canonicalForm, URIRef('http://example.org/ontolex/andi/' + entry_form_num)))
    
    try:
        morph = df[df['lemma '] == lemma]['morphology'].values[0]
    except IndexError:
        morph = ''
    if not pd.isna(morph) and morph.startswith("("):
        endings = morph.strip("()").split(", ")
        for k, ending in enumerate(endings):
            # other lexicalForm
            g.add((ontolex.LexicalEntry, ontolex.lexicalForm, URIRef('http://example.org/ontolex/andi/' + entry_form_num + '_' + str(k))))
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_form_num + '_' + str(k)), RDF.type, ontolex.Form))
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_form_num + '_' + str(k)), ontolex.writtenRep, Literal(lemma + " " + ending, lang='ani')))
    
    # lexicalForm
    g.add((URIRef('http://example.org/ontolex/andi/' + entry_form_num), RDF.type, ontolex.Form))
    g.add((URIRef('http://example.org/ontolex/andi/' + entry_form_num), ontolex.writtenRep, Literal(lemma, lang='ani')))
    ipa = df[df['lemma '] == lemma]['ipa']
    if not ipa.empty:
        ipa = ipa.values[0]
        if ipa != '':
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_form_num), ontolex.phoneticRep, Literal(ipa, lang='ipa')))
    
    # у некоторых слов есть несколько переводов, это обозначено, как новые строки таблицы
    # пока чтобы не париться я беру только один перевод
    translation = df[df['lemma '] == lemma]['meaning_ru']
    if not translation.empty:
        translation = translation.values[0]
        if isinstance(translation, str):
            translation_uri = urllib.parse.quote_plus(translation)
            
            translation = translation.replace(' ', '_').replace('/', '_')
            translation = translation.replace('\\', '_').replace('|', '_')

            ### Translation as shared reference
            #ontolex:sense :surrogate_mother_sense.
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), ontolex.sense, URIRef('http://example.org/ontolex/andi/' + 'sense_andi_' + entry_num)))

            # :surrogate_mother_sense ontolex:reference ontology:SurrogateMother.
            g.add((URIRef('http://example.org/ontolex/andi/' + 'sense_andi_' + entry_num), ontolex.reference, URIRef('http://example.org/ontolex/andi/' + translation_uri)))

            # LexicalForm of Russian word
            g.add((URIRef('http://example.org/ontolex/andi/' + 'rus_' + entry_num), RDF.type, ontolex.LexicalEntry))
            g.add((ontolex.LexicalEntry, DCTERMS.language, URIRef('http://id.loc.gov/vocabulary/iso639-2/rus')))
            g.add((ontolex.LexicalEntry, ontolex.canonicalForm, URIRef('http://example.org/ontolex/andi/' + 'rus_' + entry_form_num)))
            g.add((URIRef('http://example.org/ontolex/andi/' + 'rus_' + entry_form_num), RDF.type, ontolex.Form))
            g.add((URIRef('http://example.org/ontolex/andi/' + 'rus_' + entry_form_num), ontolex.writtenRep, Literal(translation, lang='rus')))
            g.add((URIRef('http://example.org/ontolex/andi/' + 'rus_' + entry_num), ontolex.sense, URIRef('http://example.org/ontolex/andi/' + 'sense_rus_' + entry_num)))

            # :surrogate_mother_sense ontolex:reference ontology:SurrogateMother.
            g.add((URIRef('http://example.org/ontolex/andi/' + 'sense_rus_' + entry_num), ontolex.reference, URIRef('http://example.org/ontolex/andi/' + translation_uri)))

    # partOfSpeech
    try:
        pos = df[df['lemma '] == lemma]['pos'].values[0]
    except IndexError:
        pos = ''
    if not pd.isna(pos):
        if re.match(r'.*?[nN]oun.*', pos):
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), lexinfo.partOfSpeech, lexinfo.noun))
        elif re.match(r' ?adj.*', pos):
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), lexinfo.partOfSpeech, lexinfo.adjective))
        elif re.match(r'.*?adv.*', pos):
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), lexinfo.partOfSpeech, lexinfo.adverb))
        elif re.match(r' ?verb.*', pos):
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), lexinfo.partOfSpeech, lexinfo.verb))
        elif re.match(r'num.*', pos):
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), lexinfo.partOfSpeech, lexinfo.numeral))
        elif re.match(r'.*?ord.*', pos):
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), lexinfo.partOfSpeech, lexinfo.indefiniteOrdinalNumeral))
        elif re.match(r' ?adp.*', pos):
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), lexinfo.partOfSpeech, lexinfo.adposition))
        elif re.match(r' ?gerund.*', pos):
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), lexinfo.partOfSpeech, lexinfo.gerundive))
        elif re.match(r'.*?participle.*', pos):
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), lexinfo.partOfSpeech, lexinfo.participle))
        elif re.match(r'.*?part.*', pos):
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), lexinfo.partOfSpeech, lexinfo.particle))
        elif re.match(r'excl.*', pos):
            g.add((URIRef('http://example.org/ontolex/andi/' + entry_num), lexinfo.partOfSpeech, lexinfo.exclamativePoint))
            
    i += 1

100%|██████████████████████████████████████████████████████████████████████████| 84316/84316 [2:06:24<00:00, 13.15it/s]


In [22]:
# print(g.serialize(format='turtle').decode('utf-8'))

Exception: "http://example.org/ontolex/andi/см_гьйᴴдаг|аги" does not look like a valid URI, I cannot serialize this as N3/Turtle. Perhaps you wanted to urlencode it?

In [12]:
g.serialize(destination='output_full_long2.ttl', format='turtle')

# Chukchi data

Now after we dealt with Andi data, we can move on to Chukchi data which has a slightly different format

In [7]:
from pprint import pprint

with open("chukchi-lex-common.txt", encoding='utf-8') as f:
    chukchi = f.readlines()
    
pprint(chukchi[:19])

['-lexeme\n',
 ' lex: ааңкатык\n',
 ' gramm: V\n',
 ' stem: .ааңкаты.\n',
 ' trans_ru: 1) открываться; 2) открывать рот\n',
 '\n',
 '-lexeme\n',
 ' lex: ааңкы\n',
 ' gramm: ADV\n',
 ' stem: ааңкы\n',
 ' trans_ru: открыто\n',
 '\n',
 '-lexeme\n',
 ' lex: аалёмка\n',
 ' gramm: ADV\n',
 ' stem: аалёмка//авалёмка\n',
 ' trans_ru: неслышно\n',
 '\n',
 '\n']


In [55]:
g2 = rdflib.Graph(identifier='http://example.org/ontolex/chukchi#')  # пока рандомная ссылка

In [56]:
ckt = rdflib.Namespace('http://example.org/ontolex/chukchi#')  # Корневой Namespace с рандомной ссылкой

ontolex = rdflib.Namespace('http://www.w3.org/ns/lemon/ontolex#')  # основная модель
lexinfo = rdflib.Namespace('http://www.lexinfo.net/ontology/2.0/lexinfo#')  # обозначение морфологических тегов
lime = rdflib.Namespace('http://www.w3.org/ns/lemon/lime#')
#synsem = rdflib.Namespace('http://www.w3.org/ns/lemon/synsem#')  # синтаксис
#decomp = rdflib.Namespace('http://www.w3.org/ns/lemon/decomp#')  # синтаксис
vartrans = rdflib.Namespace('http://www.w3.org/ns/lemon/vartrans#')  # для обозначения переводов и вариантов слов
skos = rdflib.Namespace('http://www.w3.org/2004/02/skos#')

In [57]:
g2.bind('ontolex', ontolex)
g2.bind('lexinfo', lexinfo)
g2.bind('dct', DCTERMS)  # Это для того, чтобы названия ISO языков грузились
g2.bind('rdfs', RDFS)
g2.bind('lime', lime)  # ??
#g2.bind('synsem', synsem)
#g2.bind('decomp', decomp)
g2.bind('vartrans', vartrans)
g2.bind('skos', skos)

g2.bind('andi', andi)

lexicon_ckt = URIRef(ckt)

In [58]:
g2.set((lexicon_ckt, RDF.type, lime.Lexicon))
g2.set((lexicon_ckt, ontolex.language, Literal("ru, ckt")))
g2.set((lexicon_ckt, RDFS.comment, Literal("lexicon-information_name:Chukchi-Russian", lang="en")))

In [59]:
# empty turtle
print(g2.serialize(format='turtle').decode('utf-8'))

@prefix andi: <http://example.org/ontolex/chukchi#> .
@prefix lime: <http://www.w3.org/ns/lemon/lime#> .
@prefix ontolex: <http://www.w3.org/ns/lemon/ontolex#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

andi: a lime:Lexicon ;
    rdfs:comment "lexicon-information_name:Chukchi-Russian"@en ;
    ontolex:language "ru, ckt" .




In [60]:
# for exploring data
i = 1
for k, line in enumerate(chukchi):
    line = line.strip('\n')
    if line.startswith("-lexeme"):
        lex = chukchi[k+1].strip(" lex: ").strip('\n')
        gramm = chukchi[k+2].strip(" gramm: ").strip('\n')
        stem = chukchi[k+3].strip(" stem: ").strip('\n')
        trans_ru = chukchi[k+4].strip(" trans_ru: ").strip('\n')
        d[i] = [lex, gramm, stem, trans_ru]
        i += 1

In [61]:
pos = []
for el in d.values():
    pos.append(el[1])
print(set(pos))

{'', 'A', 'PTH', 'PTCP', 'CONJ, ADV', 'M', 'N, A', 'INTJ', 'POSTP', 'PART', 'V', 'N, ADV', 'ADV, PART', 'NUM', 'CONJ', 'Т', 'PRED', 'PART_OF_SPEECH_UNKNOWN', 'PRON', 'N', 'Vbase', 'A, N', 'ADV', 'ADV, N'}


In [62]:
i = 1
for k, line in tqdm(enumerate(chukchi)):
    line = line.strip('\n')
    if line.startswith("-lexeme"):
        lex = chukchi[k+1].strip(" lex: ").strip('\n')
        gramm = chukchi[k+2].strip(" gramm: ").strip('\n')
        stem = chukchi[k+3].strip(" stem: ").strip('\n')
        trans_ru = chukchi[k+4].strip(" trans_ru: ").strip('\n')
        # d[i] = [lex, gramm, stem, trans_ru]
        
        entry_num = "entry_" + str(i)
        entry_form_num = "entry_" + str(i) + "_form"
        
        # LexicalEntry
        g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_num), RDF.type, ontolex.LexicalEntry))
        g2.add((ontolex.LexicalEntry, DCTERMS.language, URIRef('http://id.loc.gov/vocabulary/iso639-3/ckt')))
        g2.add((ontolex.LexicalEntry, ontolex.canonicalForm, URIRef('http://example.org/ontolex/chukchi/' + entry_form_num)))
        # written representation
        g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_form_num), RDF.type, ontolex.Form))
        g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_form_num), ontolex.writtenRep, Literal(lex, lang='ckt')))
        
        # stem (stored in otherForm)
        for k, st in enumerate(re.split(r'//|\|', stem)):
            g2.add((ontolex.LexicalEntry, ontolex.otherForm, URIRef('http://example.org/ontolex/chukchi/' + entry_form_num + '_' + str(k))))
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_form_num + '_' + str(k)), RDF.type, ontolex.Form))
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_form_num + '_' + str(k)), ontolex.writtenRep, Literal(st, lang='ckt')))
        
        # part of speech
        if gramm == 'A' or gramm == 'A, N':
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_num), lexinfo.partOfSpeech, lexinfo.adjective))
        elif gramm == 'CONJ' or gramm == 'CONJ, ADV':
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_num), lexinfo.partOfSpeech, lexinfo.conjunction))
        elif gramm == 'ADV' or gramm == 'ADV, PART' or gramm == 'ADV, N':
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_num), lexinfo.partOfSpeech, lexinfo.adverb))
        elif gramm == 'N' or gramm == 'N, A' or gramm == 'N, ADV':
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_num), lexinfo.partOfSpeech, lexinfo.noun))
        elif gramm == 'INTJ':
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_num), lexinfo.partOfSpeech, lexinfo.interjection))
        elif gramm == 'PART' or gramm == 'POSTP':
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_num), lexinfo.partOfSpeech, lexinfo.particle))
        elif gramm == 'V' or gramm == 'PRED' or gramm == 'Vbase':
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_num), lexinfo.partOfSpeech, lexinfo.verb))
        elif gramm == 'NUM':
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_num), lexinfo.partOfSpeech, lexinfo.numeral))
        elif gramm == 'PRON':
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_num), lexinfo.partOfSpeech, lexinfo.pronoun))
        
        # translation as shared reference
        # ontolex:sense :surrogate_mother_sense.
        g2.add((URIRef('http://example.org/ontolex/chukchi/' + entry_num), ontolex.sense, URIRef('http://example.org/ontolex/chukchi/' + 'sense_ckt_' + entry_num)))
        res = re.split(r'\d\)|\.', trans_ru)
        if res != []:
            for trans in res[1:]:
                translation = trans.strip(' ').replace(' ', '_').replace('/', '_').replace('\\', '_').replace(';', '_')
                
                # :surrogate_mother_sense ontolex:reference ontology:SurrogateMother.
                g2.add((URIRef('http://example.org/ontolex/chukchi/' + 'sense_ckt_' + entry_num), ontolex.reference, URIRef('http://example.org/ontolex/chukchi/' + translation)))
                
                # LexicalForm of Russian word
                g2.add((URIRef('http://example.org/ontolex/chukchi/' + 'rus_' + entry_num), RDF.type, ontolex.LexicalEntry))
                g2.add((ontolex.LexicalEntry, DCTERMS.language, URIRef('http://id.loc.gov/vocabulary/iso639-2/rus')))
                g2.add((ontolex.LexicalEntry, ontolex.canonicalForm, URIRef('http://example.org/ontolex/chukchi/' + 'rus_' + entry_form_num)))
                g2.add((URIRef('http://example.org/ontolex/chukchi/' + 'rus_' + entry_form_num), RDF.type, ontolex.Form))
                g2.add((URIRef('http://example.org/ontolex/chukchi/' + 'rus_' + entry_form_num), ontolex.writtenRep, Literal(translation, lang='rus')))
                g2.add((URIRef('http://example.org/ontolex/chukchi/' + 'rus_' + entry_num), ontolex.sense, URIRef('http://example.org/ontolex/chukchi/' + 'sense_rus_' + entry_num)))

                # :surrogate_mother_sense ontolex:reference ontology:SurrogateMother.
                g2.add((URIRef('http://example.org/ontolex/chukchi/' + 'sense_rus_' + entry_num), ontolex.reference, URIRef('http://example.org/ontolex/chukchi/' + translation)))

        else:
            translation = trans_ru.strip(' ').replace(' ', '_').replace('/', '_').replace('\\', '_').replace(';', '_')
        
            # :surrogate_mother_sense ontolex:reference ontology:SurrogateMother.
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + 'sense_ckt_' + entry_num), ontolex.reference, URIRef('http://example.org/ontolex/chukchi/' + translation)))
        
            # LexicalForm of Russian word
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + 'rus_' + entry_num), RDF.type, ontolex.LexicalEntry))
            g2.add((ontolex.LexicalEntry, DCTERMS.language, URIRef('http://id.loc.gov/vocabulary/iso639-2/rus')))
            g2.add((ontolex.LexicalEntry, ontolex.canonicalForm, URIRef('http://example.org/ontolex/chukchi/' + 'rus_' + entry_form_num)))
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + 'rus_' + entry_form_num), RDF.type, ontolex.Form))
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + 'rus_' + entry_form_num), ontolex.writtenRep, Literal(translation, lang='rus')))
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + 'rus_' + entry_num), ontolex.sense, URIRef('http://example.org/ontolex/chukchi/' + 'sense_rus_' + entry_num)))

            # :surrogate_mother_sense ontolex:reference ontology:SurrogateMother.
            g2.add((URIRef('http://example.org/ontolex/chukchi/' + 'sense_rus_' + entry_num), ontolex.reference, URIRef('http://example.org/ontolex/chukchi/' + translation)))

        
        i += 1

37697it [00:06, 5560.54it/s]


In [47]:
# test
trans_ru = "мартын (чайка)"
res = re.split(r'\d\)|\.', trans_ru)

print(res[1:])

[]


In [54]:
# test
re.split(r'//|\|', '.валвыкаңо.|вэлвыкэңут')

['.валвыкаңо.', 'вэлвыкэңут']

In [63]:
g2.serialize(destination='output_chukchi.ttl', format='turtle')

In [3]:
g_andi = rdflib.Graph()
g_andi.parse('output_translation_test.ttl')

g_ch = rdflib.Graph()
g_ch.parse('output_chukchi.ttl')

graph = g_ch + g_andi

SAXParseException: file:///D:/MyDocuments/proga/mastershse/output_translation_test.ttl:1:0: not well-formed (invalid token)